# Домашнее задание 2. Преобразование данных

Цель этого задания -- преобразовать имеющиеся атрибуты пользователей в признаки так, чтобы полученная матрица признаков была пригодна для подачи в алгоритм кластеризации. Этап конструирования признаков -- самый важный и обычно самый долгий. К нему возвращаются много раз на протяжении решения задачи анализа данных.

Кроме библиотек, использованных в первом задании, нам понадобятся следующие библиотеки:
1. [scikit-learn](http://scikit-learn.org/stable/) -- библиотека, реализующая множество алгоритмов машинного обучения и сопутствующих алгоритмов

In [54]:
import pandas as pd
import numpy as np
import pylab as pl
import sklearn.preprocessing as sp
import csv
import re
import dateutil

np.set_printoptions(linewidth=150, precision=3, suppress=True)

# Plotting config
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [55]:
ts_parser = lambda date_str: datetime.datetime.strptime(date_str, "%Y-%m") if pd.notnull(date_str) and date_str else None
df_users = pd.read_csv("hw1_out.csv", sep="\t", encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC, converters={"created_at": ts_parser}).drop('country_code', 1)
# Remove rows with users not found
df_users = df_users[pd.notnull(df_users['name'])]
#df_users["lat"].fillna(value=0, inplace=True)
#df_users["lon"].fillna(value=0, inplace=True)

In [56]:
df_users.head()

,uid,cat,name,screen_name,description,verified,location,lat,lon,country,created_at,followers_count,friends_count,statuses_count,favourites_count,listed_count
0,40585382,1,Reverend Sue,ReverendSue,Retired Interfaith Minister. Activist. Equalit...,False,United States,-98.50000,39.76000,United States,2009-05-01,7604,4282,50614,787,417
1,30005269,0,Ryan Wrasse,RWrasse,Communications Director for @SenJohnThune | Ge...,False,Capitol Hill,145.75313,15.20825,Northern Mariana Islands,2009-04-01,1920,1241,7162,127,97
2,3314758074,0,Campaign Trump,MooreJohnvii,#AlwaysTrump for President. Indp Local & Inter...,False,"Florida, USA",-82.50010,28.75054,United States,2015-08-01,24912,12404,2305,7807,217
3,14311688,0,Bernard Whitman,BernardWhitman,"Democratic strategist, pollster, and corporate...",False,New York,-74.00597,40.71427,United States,2008-04-01,1770,1298,7382,176,110
4,110457801,0,Carson Ingle,caingle,Writer for @AthleteSpeakers & @NOPACTalent. 10...,False,Orlando aka The City Beautiful,NaN,NaN,NaN,2010-02-01,1481,1189,37228,3327,76


Далее необходимо ввести новые признаки. Для каждого пользователя предлагается ввести следующие признаки:
- name_words - количество слов в имени
- screen_name_length - количество символов в псевдониме
- description_length - длина описания
- created_year - год создания аккаунта
- country_code - код страны
- verified - предлагается перевести в тип int

In [57]:
def create_new_features(df_users, features):
    # Introduce new features
    countries = list(set(df_users['country'].values))

    new_features = {"name_words":[], "screen_name_length":[], "description_length":[], "created_year":[], "country_code":[], "verified":[]}    
    for idx, user in df_users.iterrows():
        new_features["name_words"].append(len(user["name"].split()))
        new_features["screen_name_length"].append(len(user["screen_name"]))
        try:
            new_features["description_length"].append(len(user['description']))
        except:
            new_features["description_length"].append(0)
        new_features["created_year"].append(int(user['created_at'].year))
        new_features['country_code'].append(countries.index(user['country']))
        new_features['verified'].append(int(user['verified']))
        
    features.extend(new_features.keys())
    df_features = pd.DataFrame(new_features, columns = new_features.keys())
    df_features = pd.concat([df_users[df_users.columns.difference(['verified'])], df_features], axis = 1)
    
    return df_features, features

In [58]:
features = ["lat", "lon", "followers_count", "friends_count", "statuses_count", "favourites_count", "listed_count"]
df_users, features = create_new_features(df_users, features)

x = df_users[pd.notnull(df_users.cat)][features].values
y = df_users[pd.notnull(df_users.cat)]["cat"].values

In [59]:
print df_users.head()

   cat                   country created_at  \
0    1             United States 2009-05-01   
1    0  Northern Mariana Islands 2009-04-01   
2    0             United States 2015-08-01   
3    0             United States 2008-04-01   
4    0                       NaN 2010-02-01   

                                         description  favourites_count  \
0  Retired Interfaith Minister. Activist. Equalit...               787   
1  Communications Director for @SenJohnThune | Ge...               127   
2  #AlwaysTrump for President. Indp Local & Inter...              7807   
3  Democratic strategist, pollster, and corporate...               176   
4  Writer for @AthleteSpeakers & @NOPACTalent. 10...              3327   

   followers_count  friends_count        lat  listed_count  \
0             7604           4282  -98.50000           417   
1             1920           1241  145.75313            97   
2            24912          12404  -82.50010           217   
3             1770      

Посмотрим, являются ли какие-либо из выбранных признаков сильно скоррелированными. Для этого посчитаем матрицу корреляций и выберем те пары признаков, фбсолютное значения коэффициента корреляции между которыми больше 0.2. Необходимо реализовать функцию find_correlated_features, в которой нужно рассчитать коэффициенты корелляции и вывести те, которые больше 0.2. Подсказка: предлагается найти необходимую функцию в библиотеке np и реализовать find_correlated_features с использованием не более 5 строк кода.

In [60]:
def find_correlated_features(x, features):
    matrix = np.corrcoef(x)
    for i, feature_i in enumerate(features):
        for j, feature_j in enumerate(features):
            if i < j and abs(matrix[i, j]) > 0.2:
                print "Correlated features: %s + %s -> %.2f" % (feature_i, feature_j, abs(matrix[i, j]))
    

In [77]:
print x[0]
find_correlated_features(x, features)

[  -98.5      39.76      2.296     2.237     2.471     2.037     1.951     0.       11.     2009.        2.       48.      116.   ]
Correlated features: lat + lon -> 0.99
Correlated features: lat + followers_count -> 1.00
Correlated features: lat + friends_count -> 1.00
Correlated features: lat + favourites_count -> 1.00
Correlated features: lat + listed_count -> 1.00
Correlated features: lat + verified -> 1.00
Correlated features: lat + screen_name_length -> 1.00
Correlated features: lat + created_year -> 1.00
Correlated features: lat + country_code -> 1.00
Correlated features: lat + description_length -> 1.00
Correlated features: lon + followers_count -> 0.99
Correlated features: lon + friends_count -> 0.99
Correlated features: lon + favourites_count -> 0.99
Correlated features: lon + listed_count -> 0.99
Correlated features: lon + verified -> 0.99
Correlated features: lon + screen_name_length -> 0.99
Correlated features: lon + created_year -> 0.99
Correlated features: lon + country_

Выделилось 3 группы признаков:
1. Основанные на географии:  "lat", "lon", "country_code"
2. Основанные на социальной активности:  "verified", "followers_count", "friends_count", "statuses_count", "favourites_count", "listed_count", "created_year"
3. Остальные:  "name_words", "screen_name_length", "description_length"

Построим взаимные распределения пар признаков в каждой из групп, а также гистограмму значений каждого из признаков с учетом целевой переменной.

Построим попарные распределения географических признаков ([подсказка](http://anokhin.github.io/img/hw2_geo.png)).

In [62]:
from collections import Counter

def plot_two_features_scatter(ax, x_j, x_i, y):
    shuffle = 16

    g, r = (y==0).nonzero(), y.nonzero()
    
    for i in xrange(shuffle):
        ax.plot(x_i[g][i::shuffle], x_j[g][i::shuffle], 'og',
                x_i[r][i::shuffle], x_j[r][i::shuffle], 'or')
    
def plot_feature_histogram(ax, x_i, y):
    g, r = (y==0).nonzero(), y.nonzero()
    
    res_g, res_r = Counter(x_i[g]), Counter(x_i[r])
    
    for key, value in res_r.items():
        ax.bar(key, value, color = 'r', bottom = res_g[key], edgecolor = "none")
        
    for key, value in res_g.items():
        ax.bar(key, value, color = 'g', edgecolor = "none")

def plot_dataset(x, y, features):

    size = len(x[0])
    fig, axes = pl.subplots(nrows=size, ncols=size, figsize=(17,14))
    
    pl.setp(axes, xticklabels=[], yticklabels=[])
    for i in xrange(size):
        axes[size-1, i].set_xlabel(features[i])
        axes[i, 0].set_ylabel(features[i])
   
    for i in xrange(size):
        for j in xrange(size):
            if i != j:
                plot_two_features_scatter(axes[i, j], x[:, i], x[:, j], y)            
            else:
                plot_feature_histogram(axes[i, j], x[:, i], y)
                
    pl.tight_layout()
    pl.show()

In [63]:
geo_features_new = ["lat", "lon", "country_code"]
geo_features = [f for f in geo_features_new if f in features]

geo_feature_ind = [features.index(i) for i in geo_features]
#plot_dataset(x[:, geo_feature_ind], y, geo_features)

Четко видны очертания карты и то, что большинство пользователей происходят из небольшого набора стран. Если принять во внимание конечную цель -- кластеризацию пользователей -- логично предположить, что использование географических признаков для описания пользователя может оказаться не очень полезным. Причина в том, что эти признаки четко пространственно разделены (как минимум, океанами и морями). Поэтому мы рискуем вместо "интересной" кластеризации получить просто кластеры, которые будут представлять разные страны. В дальнейшем мы исключим географические признаки из рассмотрения при кластеризации пользователей.

Далее построим попарные распределения социальных признаков ([подсказка](http://anokhin.github.io/img/hw2_social1.png)).

In [64]:
social_features_new = ["verified", "followers_count", "friends_count", "statuses_count", "favourites_count", "listed_count", "created_year"]
social_features = [f for f in social_features_new if f in features]
social_feature_ind = [features.index(i) for i in social_features]
#plot_dataset(x[:, social_feature_ind], y, social_features)

Из графиков видно, что признаки "followers_count", "friends_count", "statuses_count", "favourites_count", "listed_count" сильно смещены в сторону небольших значений. В таком случае удобно сделать логарифмическое преобразрвание этих признаков, то есть применить к их значениям $x_{ij}$ функцию $\log(1 + x_{ij})$. Сделаем это и построим новые распределения ([подсказка](http://anokhin.github.io/img/hw2_social2.png)). Необходимо реализовать функцию log_transform_features, которая выполняет указанное логарифмическое преобразование.

In [70]:
def log_transform_features(data, features, transformed_features):
    feature_ind = [features.index(i) for i in transformed_features]
    data[:, feature_ind] = np.log(data[:, feature_ind] + 1)

    return data

In [66]:
transformed_features = ["followers_count", "friends_count", "statuses_count", "favourites_count", "listed_count"]
x = log_transform_features(x, features, transformed_features)

# Re-plot features
#plot_dataset(x[:, social_feature_ind], y, social_features)

Сразу бросается в глаза, что признак "verified" сильно смещен -- верифицированных пользователей очень мало. Более того, все верифицированные пользователи имеют много фолловеров, поэтому часть информации о верификации дублируется в признаке "followers_count". По этой причине в дальнейшем не будем рассмтаривать признак "verified".

После того как мы с помощью логарифмического преобразования избавились от сильной скошенности признаков, можно наблюдать некоторые интересные зависимости. Например, пользователи, имеющие много фолловеров, обязательно имеют много статусов. Следовательно, чтобы стать популярным, обязательно нужно много писать. Анализ других зависимостей остается как упражнение.

Наконец построим попарные распределения остальных признаков ([подсказка](http://anokhin.github.io/img/hw2_other.png)).

In [67]:
other_features_new = ["name_words", "screen_name_length", "description_length"]
other_features = [f for f in other_features_new if f in features]
other_feature_ind = [features.index(i) for i in other_features]
#lot_dataset(x[:, other_feature_ind], y, other_features)

Итак после первичной обработки данных мы имеем 9 числовых признаков, каждый из которых распределен в некотором своем интервале. Для того, чтобы ни один признак не получил перевеса при кластеризации, нормализуем данные так, что каждый признак распределен на отрезке $[0, 1]$. 

In [74]:
selected_features = ["followers_count", "friends_count", "statuses_count", "favourites_count", 
                     "listed_count", "created_year", "name_words", "screen_name_length", "description_length"]
selected_features_ind = [i for i, f in enumerate(features) if f in selected_features]

transformed_features = ["followers_count", "friends_count", "statuses_count", "favourites_count", "listed_count"]

#x_1 = log_transform_features(df_users[selected_features].dropna().values, selected_features, transformed_features)
x_1 = df_users[selected_features].dropna().values

#print x_1.shape
#print df_users[selected_features].dropna().values.shape
#y = df_users["cat"].values

# Replace nan with 0-s
# Is there a smarter way?
#x_1[np.isnan(x_1)] = 0

x_min = x_1.min(axis=0)
x_max = x_1.max(axis=0)
print x_max
print x_1.mean(axis=0)
x_new = (x_1 - x_min) / (x_max - x_min)

[ 71086044.    800283.    699811.    484224.    214683.      2016.        10.        15.       160.]
[ 31639.877   2084.712  14639.961   4527.66     308.517   2011.044      2.063     11.466    103.791]


Упакуем полученную матрицу в pandas DataFrame и сохраним в файл "hw2_out.csv". В следующем задании мы будем кластеризовать пользователей на оновании этих данных.

In [75]:
df_out = pd.DataFrame(data=x_new, index=df_users[selected_features + ["uid"]].dropna()["uid"], columns=[f for f in selected_features])
df_out.to_csv("hw2_out.csv", sep="\t")